# African Loan Default Prediction  
## From Raw Data to Actionable Insights

### **Objective**  
Build a machine learning model to predict loan defaults in Kenya and Ghana using:
- Customer loan histories (68,654 records)
- Macroeconomic indicators (inflation, interest rates, etc.)
  
### **Key Steps**  
1. **Data Merging**: Carefully combine loan records with economic conditions  
2. **Country-Specific Handling**: Respect differences between Kenya/Ghana economies  
3. **Missing Data Strategy**: Preserve meaningful gaps instead of artificial filling  

### **Why This Matters**  
Accurate default prediction enables:  
- Lower risk for lenders  
- Increased access to credit for borrowers  
- Stability in emerging financial markets  

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
# Load main data
train = pd.read_csv('/kaggle/input/african-credit-scoring-challenge-zindi/Train.csv')
test = pd.read_csv('/kaggle/input/african-credit-scoring-challenge-zindi/Test.csv')

# Load economic indicators
econ = pd.read_csv('/kaggle/input/african-credit-scoring-challenge-zindi/economic_indicators.csv')

In [4]:
print(f"Train: {train.shape}, Test: {test.shape}, Econ: {econ.shape}")

Train: (68654, 16), Test: (18594, 15), Econ: (27, 25)


In [5]:
print("Economic indicators columns:", econ.columns.tolist())

Economic indicators columns: ['Country', 'Indicator', 'YR2001', 'YR2002', 'YR2003', 'YR2004', 'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011', 'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021', 'YR2022', 'YR2023']


## Initial Data Observations  

### **Loan Data**  
- **Kenya (Train)**:  
  - 68,654 loans (2021-2024)  
  - 1.8% default rate (highly imbalanced)  
- **Ghana (Test)**:  
  - 18,594 loans  
  - Default rate unknown (competition test set)  

### **Economic Indicators**  
| Issue Found               | Impact                          | Our Solution                     |  
|---------------------------|----------------------------------|----------------------------------|  
| Data ends in 2023          | Can't match 2024 loans directly | Use 2023 as proxy with caution   |  
| 33% missing interest rates | Risk of biased imputation       | Preserve NAs for model to handle |  
| Ghana/Kenya differences    | Potential leakage               | Strict country separation        |  

### **Immediate Next Steps**  
1. Verify no duplicate loan IDs  
2. Check date ranges align with business logic  
3. Plot economic indicator distributions by country  

In [6]:
# Melt economic data using ACTUAL column names
econ_melted = econ.melt(
    id_vars=['Country', 'Indicator'],  # Only these exist in the file
    var_name='year_str',
    value_name='value'
)

# Extract year from strings like 'YR2021'
econ_melted['year'] = econ_melted['year_str'].str.extract('(\d+)').astype(int)
print("Melted shape:", econ_melted.shape)

Melted shape: (621, 5)


In [7]:
# Pivot indicators into columns
econ_pivoted = econ_melted.pivot_table(
    index=['Country', 'year'],
    columns='Indicator',
    values='value',
    aggfunc='first'  # Prevents duplicates
).reset_index()

# Clean column names (remove special characters)
econ_pivoted.columns = [str(col).strip().replace(' ', '_') for col in econ_pivoted.columns]
print("Pivoted shape:", econ_pivoted.shape)

Pivoted shape: (69, 11)


## Economic Data Restructuring  

### **Why This Was Necessary**  
The raw economic data came in a "wide" format that's unusable for merging:  
- Columns: `YR2001`, `YR2002`, ..., `YR2023`  
- Rows: Each indicator (inflation, interest rates, etc.) per country  

### **Critical Transformations**  
1. **Melting**: Converted years into a single column  
   ```python
   econ_melted = econ.melt(id_vars=['Country', 'Indicator'], var_name='year_str', value_name='value')

In [8]:
# Copy 2023 data to 2024
latest_data = econ_pivoted[econ_pivoted['year'] == 2023].copy()
latest_data['year'] = 2024  # Update year
econ_final = pd.concat([econ_pivoted, latest_data])

print("Final economic data shape:", econ_final.shape)
print("Years covered:", econ_final['year'].unique())

Final economic data shape: (72, 11)
Years covered: [2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]


In [9]:
# Convert dates and extract years
for df in [train, test]:
    df['disbursement_date'] = pd.to_datetime(df['disbursement_date'])
    df['year'] = df['disbursement_date'].dt.year

print("Train years:", train['year'].unique())
print("Test years:", test['year'].unique())

Train years: [2022 2024 2023 2021]
Test years: [2022 2024 2023 2021]


In [10]:
# Filter economic data (Kenya for train, Ghana for test)
econ_kenya = econ_final[econ_final['Country'] == 'Kenya'].copy()
econ_ghana = econ_final[econ_final['Country'] == 'Ghana'].copy()

print("Kenya econ:", econ_kenya.shape, "Ghana econ:", econ_ghana.shape)

Kenya econ: (24, 11) Ghana econ: (24, 11)


## Safe Merging Architecture  

### **The Risk**  
Economic data has one row per country-year, but loans have multiple records per year.  
A naive merge could explode duplicates (68K loans × 24 years = 1.6M rows!).  

### **The Solution**  
1. **Country Filtering**:  
   ```python
   econ_kenya = econ_final[econ_final['Country']=='Kenya']
   econ_ghana = econ_final[econ_final['Country']=='Ghana']

In [11]:
# Merge train with Kenya econ data
train_merged = train.merge(
    econ_kenya,
    left_on=['year'],
    right_on=['year'],
    how='left',
    validate='many_to_one'  # Ensures no row multiplication
)

# Merge test with Ghana econ data
test_merged = test.merge(
    econ_ghana,
    left_on=['year', 'country_id'],
    right_on=['year', 'Country'],
    how='left',
    validate='many_to_one'
)

print("Train merged:", train_merged.shape, "Test merged:", test_merged.shape)

Train merged: (68654, 27) Test merged: (18594, 26)


In [12]:
# Check for duplicates
assert train.shape[0] == train_merged.shape[0], "Train rows changed!"
assert test.shape[0] == test_merged.shape[0], "Test rows changed!"

# Check NA rates
print("NA counts in train:", train_merged.isna().sum().sum())
print("NA counts in test:", test_merged.isna().sum().sum())

NA counts in train: 137269
NA counts in test: 168296


## The Missing Data Dilemma  

### **What We Found**  
- **161,029 NAs** in training, **168,296** in testing  
- Concentrated in:  
  - Interest rate spreads (33% missing)  
  - Lending rates (33% missing)  

### **Decision Process**  
| Option | Pros | Cons |  
|--------|------|------|  
| **Keep NAs** | Honest representation | Requires NA-aware models |  
| **Median Fill** | Works with all models | Distorts distributions |  
| **Indicator Columns** | Captures missingness | Increases dimensionality |  

### **Final Choice**  
Preserve NAs because:  
1. XGBoost handles them natively via `enable_categorical=True`  
2. Missingness often correlates with default risk in finance  
3. Avoids making assumptions about economic conditions  

In [13]:
# Forward-fill 2024 economic data from 2023 values 
econ_final = econ_final.sort_values(['Country', 'year'])
econ_final = econ_final.groupby('Country').ffill()  # Propagates last known values

# Re-do the merges with updated economic data
train_merged = train.merge(econ_kenya, on='year', how='left', validate='many_to_one')
test_merged = test.merge(econ_ghana, left_on=['year','country_id'], right_on=['year','Country'], how='left', validate='many_to_one')

In [14]:
print("Remaining NAs in Train:", train_merged.isna().sum().sum())
print("Remaining NAs in Test:", test_merged.isna().sum().sum())

Remaining NAs in Train: 137269
Remaining NAs in Test: 168296


In [15]:
# Check pre-fill NA counts by indicator
print("Original NA counts in economic data:")
print(econ_final.isna().sum())

Original NA counts in economic data:
year                                                          0
Average_precipitation_in_depth_(mm_per_year)                  0
Deposit_interest_rate_(%)                                     4
Fossil_fuel_energy_consumption_(%_of_total)                   0
Inflation,_consumer_prices_(annual_%)                         0
Interest_rate_spread_(lending_rate_minus_deposit_rate,_%)    28
Lending_interest_rate_(%)                                    28
Official_exchange_rate_(LCU_per_US$,_period_average)          0
Real_interest_rate_(%)                                       28
Unemployment_rate                                             0
dtype: int64


In [16]:
print("Columns in econ_final:", econ_final.columns.tolist())

Columns in econ_final: ['year', 'Average_precipitation_in_depth_(mm_per_year)', 'Deposit_interest_rate_(%)', 'Fossil_fuel_energy_consumption_(%_of_total)', 'Inflation,_consumer_prices_(annual_%)', 'Interest_rate_spread_(lending_rate_minus_deposit_rate,_%)', 'Lending_interest_rate_(%)', 'Official_exchange_rate_(LCU_per_US$,_period_average)', 'Real_interest_rate_(%)', 'Unemployment_rate']


In [17]:
# Melt and clean year 
econ_melted = econ.melt(
    id_vars=['Country', 'Indicator'],
    var_name='year_str',
    value_name='value'
)
econ_melted['year'] = econ_melted['year_str'].str.extract('(\d+)').astype(int)

In [18]:
# Pivot with explicit handling
econ_final = econ_melted.pivot(
    index=['Country', 'year'],
    columns='Indicator',
    values='value'
).reset_index()

In [19]:
print("NA counts before filling:", econ_final.isna().sum())

NA counts before filling: Indicator
Country                                                       0
year                                                          0
Average precipitation in depth (mm per year)                  6
Deposit interest rate (%)                                    10
Fossil fuel energy consumption (% of total)                  27
Inflation, consumer prices (annual %)                         0
Interest rate spread (lending rate minus deposit rate, %)    33
Lending interest rate (%)                                    33
Official exchange rate (LCU per US$, period average)          0
Real interest rate (%)                                       33
Unemployment rate                                             0
dtype: int64


In [20]:
print("Actual columns in econ_final:")
print(econ_final.columns.tolist())

Actual columns in econ_final:
['Country', 'year', 'Average precipitation in depth (mm per year)', 'Deposit interest rate (%)', 'Fossil fuel energy consumption (% of total)', 'Inflation, consumer prices (annual %)', 'Interest rate spread (lending rate minus deposit rate, %)', 'Lending interest rate (%)', 'Official exchange rate (LCU per US$, period average)', 'Real interest rate (%)', 'Unemployment rate']


In [21]:
# 1. Simple merge without any NA filling
train_merged = train.merge(
    econ_final[econ_final['Country']=='Kenya'],
    on='year',
    how='left',
    validate='many_to_one'
)

test_merged = test.merge(
    econ_final[econ_final['Country']=='Ghana'],
    left_on=['year', 'country_id'],
    right_on=['year', 'Country'],
    how='left',
    validate='many_to_one'
)

# 2. Save as-is
train_merged.to_csv('train_merged_RAW.csv', index=False)
test_merged.to_csv('test_merged_RAW.csv', index=False)

# 3. Quick verification
print("Final NA counts (natural):")
print("Train:", train_merged.isna().sum().sum())
print("Test:", test_merged.isna().sum().sum())

Final NA counts (natural):
Train: 161029
Test: 168296


In [22]:
# Save with clear naming
train_merged.to_csv('train_merged_kenya.csv', index=False)
test_merged.to_csv('test_merged_ghana.csv', index=False)

print("Files saved successfully!")
print("Final shapes - Train:", train_merged.shape, "Test:", test_merged.shape)

Files saved successfully!
Final shapes - Train: (68654, 27) Test: (18594, 26)


In [23]:
import pandas as pd

# Load your final merged files from yesterday
train = pd.read_csv('/kaggle/working/train_merged_RAW.csv')
test = pd.read_csv('/kaggle/working/test_merged_RAW.csv')

print(f"Train shape: {train.shape}, Test shape: {test.shape}")
print("NA counts - Train:", train.isna().sum().sum(), "Test:", test.isna().sum().sum())

Train shape: (68654, 27), Test shape: (18594, 26)
NA counts - Train: 161029 Test: 168296


In [24]:
# Modern models like XGBoost handle NAs automatically
train.to_csv('final_train.csv', index=False)
test.to_csv('final_test.csv', index=False)

# Final Project Summary

## Outcomes Achieved
1. Clean merge of loan + economic data
   - Zero duplicate rows created
   - All country data kept separate
2. Missing values handled transparently
   - 161,029 NAs preserved in train
   - 168,296 NAs preserved in test

## Core Lessons
- African financial data requires:
  * Country-specific treatment
  * Careful date alignment
  * Respect for missing data

## Next Steps
See separate EDA notebook for:
- Default rate distributions
- Economic indicator analysis
- Missing value patterns

"Good data merging enables great analysis."